# Week 3
## 1. Scraping data from wikipedia

!pip install -q beautifulsoup4

!pip install -q lxml

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
content = requests.get(url)
content = BeautifulSoup(content.content, 'lxml')

In [4]:
table_html = content.select('.wikitable tbody tr')
table = [row.select('td,th') for row in table_html]
table = list(map(lambda line: [element.get_text().replace("\n","") for element in line], table))

In [5]:
table[:5]

[['Postcode', 'Borough', 'Neighbourhood'],
 ['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village']]

## Data cleaning

In [6]:
import pandas as pd

In [7]:
data = pd.DataFrame(table[1:], columns=table[0])

In [8]:
data.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Queen's Park,Not assigned


### 1.1 Displaying repeated postal codes

In [9]:
l = data.groupby('Postcode').count().sort_values('Borough', ascending=False)
l = l[l.Neighbourhood > 1]

In [10]:
l.head()

,Borough,Neighbourhood
Postcode,,
M8Y,8,8
M9V,8,8
M5V,7,7
M4V,5,5
M9B,5,5


In [11]:
l.shape

(56, 2)

### 1.2 Dropping repeated values and creating a new line with the names of the concatenated neighborhoods

In [12]:
data = data[data['Borough'] != 'Not assigned']
data['Neighbourhood'] = data.apply(lambda row: row[1] if row[2] == 'Not assigned' else row[2], axis=1)

In [13]:
for code in l.index:
    mask = data['Postcode'] == code
    b = list(data[mask].Borough)[-1]
    n = ', '.join(list(data[mask].Neighbourhood))
    data = data[~mask]
    s =  pd.Series({'Postcode':code, 'Borough':b, 'Neighbourhood':n})
    data = data.append(s, ignore_index=True) 

### 1.3 Result

In [14]:
l_a = data.groupby('Postcode').count().sort_values('Borough', ascending=False)
l_a = l_a[l_a.Neighbourhood > 1]
l_a.shape

(0, 2)

In [15]:
data = data.sort_values('Postcode').reset_index(drop = True)
data.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [16]:
data.shape

(103, 3)

## 2. Getting latitude and longitude of each Neighbourhood

In [17]:
latlon = pd.read_csv('Geospatial_Coordinates.csv')

In [18]:
latlon.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [19]:
(latlon['Postal Code'] == data['Postcode']).unique()

array([ True])

In [20]:
data['Latitude'] = latlon['Latitude']
data['Longitude'] = latlon['Longitude']

In [21]:
data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [22]:
data.shape

(103, 5)

## 3. Generating map with the neighbourhoods

In [23]:
!pip install -q folium

In [24]:
import folium
m = folium.Map(location=[43.651070, -79.347015], zoom_start=12)
for post, borough, neighbourhood, lat, lon in data.values:
    folium.Marker(
        location=[lat, lon],
        radius=10,
        popup=post,
        color='#3186cc',
        fill=True,
        fill_color='#3186cc'
    ).add_to(m)

m

## 4. Using the Foursquare api

In [56]:
CLIENT_ID = '4JY5FPLSHJLBIQBJ1TAD3JNQJKJBFVTGCMK3KBL0UJIFS0AT'
CLIENT_SECRET = 'O1GB1D10JDUEVA13JSU3XKYIWJTAL5M11ZYT0TK4FEPU3SAI'
VERSION = '20180605'

Search the 100 vienues next to each neighbourhood

In [26]:
import json
from pandas.io.json import json_normalize

In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT = 100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
toronto_venues = getNearbyVenues(
    data['Neighbourhood'], 
    data['Latitude'], 
    data['Longitude']
)

In [29]:
toronto_venues.shape

(2237, 7)

In [30]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",9,9,9,9,9,9
"Alderwood, Long Branch",11,11,11,11,11,11
"Bathurst Manor, Downsview North, Wilson Heights",21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",24,24,24,24,24,24
Berczy Park,56,56,56,56,56,56


In [31]:
mask = toronto_venues['Venue Category'] == 'Neighborhood'
toronto_venues = toronto_venues[~mask]

In [32]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 269 uniques categories.


In [33]:
one_hot_toronto = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
one_hot_toronto['Neighborhood'] = toronto_venues['Neighborhood']

In [34]:
fixed_columns = [one_hot_toronto.columns[-1]] + list(one_hot_toronto.columns[:-1])
one_hot_toronto = one_hot_toronto[fixed_columns]

In [35]:
one_hot_toronto.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
one_hot_toronto.shape

(2233, 270)

In [37]:
toronto_grouped = one_hot_toronto.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.020202,...,0.020202,0.0,0.0,0.0,0.0,0.010101,0.0,0.0,0.010101,0.0
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
4,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0


In [38]:
toronto_grouped.shape

(100, 270)

In [39]:
def top_5(row):
    ordered = row[1:].sort_values(ascending=False).index.values[:5]
    response = {'Neighborhood': row[0]}
    for i in range(5):
        response.update({'Top {}'.format(i + 1): ordered[i]})
    return response

In [40]:
toronto_neighborhood_top5 = pd.DataFrame.from_records(toronto_grouped.apply(top_5, 1).values)

## 5. Cluster Neighborhoods

In [41]:
from sklearn.cluster import KMeans

In [42]:
toronto_clustering = toronto_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=5, random_state=0).fit(toronto_clustering)
kmeans.labels_[0:10]

array([1, 1, 2, 0, 0, 1, 1, 1, 1, 1])

In [43]:
toronto_neighborhood_top5.insert(0, 'Cluster Labels', kmeans.labels_)

In [44]:
toronto_neighborhood_top5.head()

,Cluster Labels,Neighborhood,Top 1,Top 2,Top 3,Top 4,Top 5
0,1,"Adelaide, King, Richmond",Coffee Shop,Café,Thai Restaurant,Bar,Cosmetics Shop
1,1,Agincourt,Lounge,Breakfast Spot,Latin American Restaurant,Chinese Restaurant,Yoga Studio
2,2,"Agincourt North, L'Amoreaux East, Milliken, St...",Playground,Park,Yoga Studio,Donut Shop,Dessert Shop
3,0,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pharmacy,Pizza Place,Fast Food Restaurant,Coffee Shop
4,0,"Alderwood, Long Branch",Pizza Place,Pool,Pub,Coffee Shop,Pharmacy


In [45]:
data_merged = data.join(toronto_neighborhood_top5.set_index('Neighborhood'), on='Neighbourhood')

In [46]:
data_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,Top 1,Top 2,Top 3,Top 4,Top 5
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,1.0,Fast Food Restaurant,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,1.0,History Museum,Bar,Yoga Studio,Drugstore,Diner
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1.0,Mexican Restaurant,Spa,Pizza Place,Electronics Store,Breakfast Spot
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Korean Restaurant,Dumpling Restaurant,Diner,Discount Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Hakka Restaurant,Caribbean Restaurant,Bank,Fried Chicken Joint,Thai Restaurant


In [47]:
mask = data_merged['Cluster Labels'] >= 0
data_merged = data_merged[mask]

In [48]:
m = folium.Map(location=[43.651070, -79.347015], zoom_start=12)

for post, neighbourhood, lat, lon, cluster in zip(data_merged['Postcode'].values, 
                                                           data_merged['Neighbourhood'].values, 
                                                           data_merged['Latitude'].values, 
                                                           data_merged['Longitude'].values,
                                                           data_merged['Cluster Labels'].values):
    colors = ['#3186cc', '#cc9900', '#99cc18', '#18cc99', '#cc0086']
    folium.CircleMarker(
        location=[lat, lon],
        radius=10,
        popup=neighbourhood + ' class: {}'.format(cluster),
        color=colors[int(cluster)],
        fill=True,
        fill_color= colors[int(cluster)]
    ).add_to(m)
m

## 6. Analise the clusters

### 6.1 Cluster 2

The cluster 2 neighborhoods has in the top 3 Parks, witch seems their similarity among.

In [54]:
data_merged[data_merged['Cluster Labels'] == 2]

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,Top 1,Top 2,Top 3,Top 4,Top 5
14,M1V,Scarborough,"Agincourt North, L'Amoreaux East, Milliken, St...",43.815252,-79.284577,2.0,Playground,Park,Yoga Studio,Donut Shop,Dessert Shop
23,M2P,North York,York Mills West,43.752758,-79.400049,2.0,Park,Bank,Bar,Convenience Store,Yoga Studio
30,M3K,North York,"CFB Toronto, Downsview East",43.737473,-79.464763,2.0,Airport,Park,Snack Place,Yoga Studio,Donut Shop
40,M4J,East York,East Toronto,43.685347,-79.338106,2.0,Convenience Store,Park,Dumpling Restaurant,Diner,Discount Store
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,2.0,Tennis Court,Park,Playground,Restaurant,Dog Run
50,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,2.0,Park,Trail,Playground,Yoga Studio,Dessert Shop
73,M6C,York,Humewood-Cedarvale,43.693781,-79.428191,2.0,Field,Trail,Park,Hockey Arena,Dim Sum Restaurant
74,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512,2.0,Park,Market,Pool,Yoga Studio,Donut Shop
90,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944,2.0,Park,River,Smoke Shop,Yoga Studio,Doner Restaurant
98,M9N,York,Weston,43.706876,-79.518188,2.0,Convenience Store,Park,Dumpling Restaurant,Diner,Discount Store


The others clusters be similarity of this